In [ ]:
from langchain_groq import ChatGroq
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.vectorstores import Chroma ##nosql
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter #tokens
import os
import gradio as gr

def initialize_llm():
  llm=ChatGroq(
      temperature=0,
      groq_api_key="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",
      model="llama-3.3-70b-versatile"
  )
  return llm

def create_vector_DB():
  loader=DirectoryLoader("/content/documentdata/", glob="*.pdf", loader_cls=PyPDFLoader)
  docs=loader.load()
  text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
  text=text_splitter.split_documents(docs)
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
  vector_db=Chroma.from_documents(text,embeddings,persist_directory='./chroma_db')
  vector_db.persist()
  print('ChromaDB created and data saved')
  return vector_db

def setup_qa_chain(vector_db,llm):
  retriever=vector_db.as_retriever()
  prompt_templates="""You are a compassionate mental health chatbot. Respond thoughtfully to the following
  question
  {context}
  User: {question}
  Chatbot: """
  PROMPT=PromptTemplate(template=prompt_templates, input_variables=['context','question'])
  qa_chain=RetrievalQA.from_chain_type(
      llm=llm,
      chain_type="stuff",
      retriever=retriever,
      chain_type_kwargs={"prompt":PROMPT}
  )
  return qa_chain

print("Initializing Menty...")
llm=initialize_llm()
db_path="/content/chroma_db"
if not os.path.exists(db_path):
  vector_db=create_vector_DB()
else:
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
  vector_db=Chroma(persist_directory=db_path, embedding_function=embeddings)
qa_chain=setup_qa_chain(vector_db, llm)

def chatbot_response(user_input, history):
    if history is None:
        history = []  # Ensure history is a list

    if not user_input.strip():
        return history,""  # Ignore empty input

    response = qa_chain.run(user_input)  # Use actual model response

    history.append([user_input, response])  # Ensure it's a list, not tuple

    return history,"" # Return updated chat history

# Define the chat UI
with gr.Blocks() as demo:
    gr.Markdown("## 🧠 Mental Health Chatbot\nA compassionate chatbot to help with mental health queries.")

    chatbot = gr.Chatbot()  # Chat UI
    user_input = gr.Textbox(placeholder="Type your message here...")

    user_input.submit(chatbot_response, [user_input, chatbot],  [chatbot, user_input])

demo.launch(share=True)

Initializing Menty...


<ipython-input-3-550c3923068d>:24: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or dat

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

<ipython-input-3-550c3923068d>:26: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vector_db.persist()
<ipython-input-3-550c3923068d>:73: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()  # Chat UI


ChromaDB created and data saved
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fdf77f525379de4851.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
